<a href="https://colab.research.google.com/github/sofials2002/SOFIA/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wage Prediction using Machine Learning

In this notebook we will answer what determines the wage of workers from a predictive perspective.

This example focuses on a sample of Registered Nurses in the US collected during 2017. The hourly wage of a nurse is denoted by $Y$ and $X$ is a vector of nurses' characteristics, e.g., human capital, demographics, job-relevant characteristics. The question that we want to answer is:

- How to use nurses' characteristics, such as education and experience, to best predict wages?


In [1]:
import numpy as np
import pandas as pd
import patsy
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import TransformerMixin, RegressorMixin, BaseEstimator
from sklearn.pipeline import make_pipeline

import warnings
warnings.simplefilter('ignore')

## Data

In [2]:
# Read data
path_data = "https://github.com/pabloestradac/causalml-basics/raw/main/data/"
df = pd.read_csv(path_data + 'wages_nurses.csv')
df.tail()

,lwage,female,age,race,children,marital,education,RN_experience,left_nursing,country_ed_US,english_only,military,certificates,labor_union,work_setting,work_situation,level_care,care_specialty,state
5617,3.496087,1,72,white,0,widowed_divorced_sep,ed_assoc,51,left_0,1,1,never_served_mil,0,0,SET_long_term_inpatient,SIT_self_employed,LC_nursing_home,CC_no_patient_care,state_n13
5618,3.678408,1,50,white,0,never_married,ed_msn,14,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_education,CC_emergency_care,state_n24
5619,3.952845,0,36,white,0,currently_married,ed_bsn,4,left_0,1,1,never_served_mil,2,1,SET_hospital,SIT_agency_facility,LC_inpatient,CC_oncology,state_n32
5620,3.957960,1,61,white,0,currently_married,ed_bsn,40,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_inpatient,CC_no_patient_care,state_n37
5621,3.100270,1,38,white,0,currently_married,ed_assoc,3,left_0,1,1,never_served_mil,2,0,SET_hospital,SIT_agency_facility,LC_others,CC_medical_surgical,state_n29


In [3]:
# Outcome and regressors
y = np.log(df['lwage']).values
Z = df.drop(['lwage'], axis=1)
Z.shape

(5622, 18)

Construct a prediction rule for hourly (log) wage $y$, which depends linearly on relevant characteristics $Z$:

$$
y = g(Z) + e
$$

Then, assess the predictive performance of a given model using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$. Thus, we measure the prediction quality of the models via data splitting.


In [5]:
# Obtain train_idx and test_idx to divide y_train and y_test
train_idx, test_idx = train_test_split(np.arange(len(y)), test_size=0.3, random_state=42) #creates an array of integers from 0 to len(y)-1.
#these are then split into training and testing indices.

y_train, y_test = y[train_idx], y[test_idx] #it's good to have indexes because we'll be applying a lot of transformations to this data.

## Linear Models

We employ two different specifications for prediction:

1. Basic Model: $X$ consists of a set of raw regressors

2. Flexible Model: $X$ consists of all raw regressors from the basic model plus a dictionary of transformations.


In [6]:
model_base = ('0 + female + age + race + children + marital '
              '+ education + RN_experience + RN_experience**2 + RN_experience**3 + RN_experience**4 '
              '+ left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state')
#we add the zero because with scikit learn we don't want an intercept.
Zbase = patsy.dmatrix(model_base, Z, return_type='dataframe').values
X_train, X_test = Zbase[train_idx], Zbase[test_idx]

In [7]:
def metrics(X_test, y_test, estimator):
    mse = np.mean((y_test - estimator.predict(X_test))**2)
    semse = np.std((y_test - estimator.predict(X_test))**2) / np.sqrt(len(y_test))
    r2 = 1 - mse / np.var(y_test)
    print(f'{mse:.4f}, {semse:.4f}, {r2:.4f}')
    return mse, semse, r2

results = {}

Let's start by running a simple OLS regression.

In [8]:
# OLS using LinearRegression
lr = LinearRegression()
ypred_ols = lr.fit(X_train, y_train).predict(X_test) #always use X_test for prediction.
results['ols'] = metrics(X_test, y_test, lr)

0.0079, 0.0005, 0.2313


In [9]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Lasso
lcv = make_pipeline(StandardScaler(), LassoCV(cv=cv, random_state=42)) #we use LASSO cv, because otherwise we'd have to define a Lambda.
lcv.fit(X_train, y_train) #we fit the lasso model on the training set.
ypred_lcv = lcv.predict(X_test) #now we predict.
results['lassocv'] = metrics(X_test, y_test, lcv)

0.0078, 0.0005, 0.2321


In [12]:
# Ridge
rcv = make_pipeline(StandardScaler(), RidgeCV(cv=cv))
rcv.fit(X_train, y_train)
ypred_rcv = rcv.predict(X_test)
results['ridgecv'] = metrics(X_test, y_test, rcv)

0.0078, 0.0005, 0.2316


In [13]:
# Elastic Net
ecv = make_pipeline(StandardScaler(), ElasticNetCV(cv=cv, random_state=42))
ecv.fit(X_train, y_train)
ypred_ecv = ecv.predict(X_test)
results['elanetcv'] = metrics(X_test, y_test, ecv)

0.0078, 0.0005, 0.2321


In [14]:
X_train.shape

(3935, 96)

Use theoretical-optimal $\lambda$ for lasso.

This is a based on a Python implementation made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is this [repository](https://github.com/maxhuppertz/hdmpy).

Run the following code to install the library:

```python
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install multiprocess
```

In [16]:
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install multiprocess

Cloning into 'hdmpy'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 70 (delta 39), reused 52 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 25.30 KiB | 2.53 MiB/s, done.
Resolving deltas: 100% (39/39), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.1 MB/s eta 0:00:00


In [17]:
import hdmpy #python version of r's HDM (high dimensional models)

# Wrap the package so that it has the familiar sklearn API
class RLasso(BaseEstimator, RegressorMixin):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    @property
    def coef_(self):
        return np.array(self.rlasso_.est['beta']).flatten()

    @property
    def intercept_(self):
        return np.array(self.rlasso_.est['intercept'])

    def predict(self, X):
        return X @ self.coef_ + self.intercept_

In [19]:
# Optimal lasso
lasso = make_pipeline(StandardScaler(), RLasso(post=False))
lasso.fit(X_train, y_train)
ypred_lasso = lasso.predict(X_test)
results['lasso'] = metrics(X_test, y_test, lasso)

0.0082, 0.0005, 0.2006


In [20]:
# Optimal post-lasso
postlasso = make_pipeline(StandardScaler(), RLasso(post=True))
postlasso.fit(X_train, y_train)
ypred_postlasso = postlasso.predict(X_test)
results['postlasso'] = metrics(X_test, y_test, postlasso)

0.0084, 0.0006, 0.1749


In [22]:
pd.DataFrame(results).T #by looking at this table we can see that every model performs kind of the same as OLS.

,0,1,2
ols,0.007852,0.000513,0.231310
lassocv,0.007843,0.000515,0.232147
ridgecv,0.007849,0.000513,0.231600
elanetcv,0.007844,0.000515,0.232127
lasso,0.008165,0.000533,0.200628
postlasso,0.008428,0.000556,0.174893


## Non-Linear Models

Apply regression trees, random forests, boosted trees and neural nets to estimate the regression function $g(Z)$.

In [24]:
dtr = DecisionTreeRegressor(min_samples_leaf=5)
dtr.fit(X_train, y_train)
ypred_dtr = dtr.predict(X_test)
results['trees'] = metrics(X_test, y_test, dtr)

0.0108, 0.0006, -0.0608


In [25]:
rf = RandomForestRegressor(n_estimators=5000, min_samples_leaf=5)
rf.fit(X_train, y_train)
ypred_rf = rf.predict(X_test)
results['randforest'] = metrics(X_test, y_test, rf)

0.0082, 0.0005, 0.2021


In [ ]:
gbf =
gbf.fit(X_train, y_train)
ypred_gbf = gbf.predict(X_test)
results['gradboost'] = metrics(X_test, y_test, gbf)

In [ ]:
nnet =
ypred_nnet =
results['neuralnet'] = metrics(X_test, y_test, nnet)

In [ ]:
import skorch
from torch import nn, optim

arch = nn.Sequential(nn.Linear(X_train.shape[1], 200), nn.ReLU(),
                     nn.Linear(200, 20), nn.ReLU(),
                     nn.Linear(20, 1))
nnet_early = skorch.NeuralNetRegressor(arch, lr=0.01, batch_size=10,
                                       max_epochs=100,
                                       optimizer=optim.Adam,
                                       callbacks=[skorch.callbacks.EarlyStopping()])
nnet_early.fit(X_train.astype(np.float32), y_train.reshape(-1, 1).astype(np.float32))
ypred_nnet_early = nnet_early.predict(X_test.astype(np.float32)).flatten()
results['neuralnet_early'] = metrics(X_test.astype(np.float32),
                                y_test.reshape(-1, 1).astype(np.float32), nnet_early)

In [ ]:
df = pd.DataFrame(results).T
df.columns = ['MSE', 'S.E. MSE', 'Rsq']
df.sort_values('MSE', ascending=True)

## Stacking

In the final step, build a prediction model by combining the strength of each model.
$$
g(z) = \sum_{k=1}^K \alpha_k g_k(z)
$$
where $g_k$'s denote our prediction rules from the table above and the $\alpha_k$'s are the corresponding weights.

In [ ]:
method_name = ['OLS', 'CV Lasso', 'CV Ridge', 'CV ElasticNet', 'Lasso', 'Post-Lasso OLS',
               'Decision Tree', 'Random Forest', 'Boosted Forest', 'Neural Net', 'Neural Net (early stopping)']
ypreds =
stack_ols =
pd.DataFrame({'weight': stack_ols.coef_}, index=method_name).round(2)

In [ ]:
# Calculate the test sample MSE and Rsq
# We should have left out a third sample to validate the performance of the stacked model
mse = np.mean((y_test - stack_ols.predict(ypreds))**2)
r2 = 1 - mse / np.var(y_test)
print(f'MSE: {mse:.4f}, R^2: {r2:.4f}')

## Pipelines

We can also do it in a more sklearn way, by defining a formula transformer and corresponding pipelines

In [ ]:
class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula):
        self.formula = formula

    def fit(self, X, y=None):
        mat = patsy.dmatrix(self.formula, X, return_type='matrix')
        self.design_info = mat.design_info
        return self

    def transform(self, X, y=None):
        return patsy.build_design_matrices([self.design_info], X)[0]

In [ ]:
base = FormulaTransformer(model_base)
methods =

In [ ]:
results = {}
ypreds = np.zeros((len(test_idx), len(methods)))  # test predictions used for stacking

for it, (name, estimator) in enumerate(methods):
    estimator.fit(Z.iloc[train_idx], y[train_idx])
    results[name] = metrics(Z.iloc[test_idx], y[test_idx], estimator)
    ypreds[:, it] = estimator.predict(Z.iloc[test_idx])

In [ ]:
df = pd.DataFrame(results).T
df.columns = ['MSE', 'S.E. MSE', 'Rsq']
df

1. Partition the data in k-folds

2. For each fold, train each of the estimators in the `methods` parameter on all the data outside of the fold and then predict on the data in the fold

3. Train a `final_estimator` predicting the true outcome using the out-of-fold predictions of each method as features

4. All the base estimators are re-fitted on all the data and the final predictor will first predict based on each fitted based estimator and then aggregate based on the fitted `final_estimator`


In [ ]:
# Stacking with sklearn
stack =

In [ ]:
# Weights of the final estimator
pd.DataFrame({'weight': stack.final_estimator_.coef_}, index=[name for name, _ in methods])

In [ ]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], stack)

## AutoML

In [ ]:
from flaml import AutoML

automl =

In [ ]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], automl)

In [ ]:
# Stacking with flaml
automl =

In [ ]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], automl)